In [1]:
#Import standard python modules
import os.path
import sys

#Determine local directory path:
local_path = os.path.abspath('') # os.path.dirname(os.path.abspath(__file__))
print(f"Local path appears to be {local_path}")

#set path to ADF lib:
lib_path = os.path.join(local_path,"lib")
print(f"Lib path appears to be {lib_path}")

#set path to ADF plotting scripts directory:
plotting_scripts_path = os.path.join(local_path, "scripts", "plotting")
print(plotting_scripts_path)

#Add paths to python path:
sys.path.append(lib_path)
sys.path.append(plotting_scripts_path)

#Import ADF diagnostics object:
from adf_diag import AdfDiag

Local path appears to be /glade/u/home/brianpm/Code/CAM_diagnostics
Lib path appears to be /glade/u/home/brianpm/Code/CAM_diagnostics/lib
/glade/u/home/brianpm/Code/CAM_diagnostics/scripts/plotting


In [14]:
from pathlib import Path
config_file = Path.home()/"drive_diagnostics"/"test_variable_prefs.yaml"

#Set path to config YAML file:
# config_fil = "/Users/brianpm/Code/scratch/CAM_diagnostics/config_user_module_example.yaml"

#Initialize ADF object:
adf = AdfDiag(config_file)

In [15]:
basic_info_dict = adf.read_config_var("diag_basic_info")
print(basic_info_dict)

baseline_dict = adf.read_config_var("diag_cam_baseline_climo")
print(baseline_dict)

{'compare_obs': False, 'create_html': True, 'obs_climo_loc': '/glade/work/brianpm/observations/climo_files', 'cam_regrid_loc': '/glade/scratch/brianpm/cam_diag_regrid_files', 'cam_overwrite_regrid': True, 'cam_diag_plot_loc': '/glade/scratch/brianpm/cam_diag_plots', 'use_defaults': True}
{'calc_cam_climo': True, 'cam_overwrite_climo': True, 'cam_case_name': 'b.e20.BHIST.f09_g16.20thC.125.02', 'cam_hist_loc': '/glade/p/cesm/ADF/b.e20.BHIST.f09_g16.20thC.125.02', 'cam_climo_loc': '/glade/scratch/brianpm/cam_diag_climo/files/b.e20.BHIST.f09_g16.20thC.125.02', 'start_year': 1990, 'end_year': 1999, 'cam_ts_done': False, 'cam_ts_save': True, 'cam_overwrite_ts': False, 'cam_ts_loc': '/glade/scratch/brianpm/cam_diag_ts_files'}


In [16]:
## WHERE ARE THE CLIMO FILES:
list(Path(baseline_dict['cam_climo_loc']).glob(baseline_dict['cam_case_name']+"*"))

[PosixPath('/glade/scratch/brianpm/cam_diag_climo/files/b.e20.BHIST.f09_g16.20thC.125.02/b.e20.BHIST.f09_g16.20thC.125.02_PRECL_climo.nc'),
 PosixPath('/glade/scratch/brianpm/cam_diag_climo/files/b.e20.BHIST.f09_g16.20thC.125.02/b.e20.BHIST.f09_g16.20thC.125.02_U_climo.nc'),
 PosixPath('/glade/scratch/brianpm/cam_diag_climo/files/b.e20.BHIST.f09_g16.20thC.125.02/b.e20.BHIST.f09_g16.20thC.125.02_PS_climo.nc'),
 PosixPath('/glade/scratch/brianpm/cam_diag_climo/files/b.e20.BHIST.f09_g16.20thC.125.02/b.e20.BHIST.f09_g16.20thC.125.02_RELHUM_climo.nc'),
 PosixPath('/glade/scratch/brianpm/cam_diag_climo/files/b.e20.BHIST.f09_g16.20thC.125.02/b.e20.BHIST.f09_g16.20thC.125.02_SWCF_climo.nc'),
 PosixPath('/glade/scratch/brianpm/cam_diag_climo/files/b.e20.BHIST.f09_g16.20thC.125.02/b.e20.BHIST.f09_g16.20thC.125.02_LWCF_climo.nc'),
 PosixPath('/glade/scratch/brianpm/cam_diag_climo/files/b.e20.BHIST.f09_g16.20thC.125.02/b.e20.BHIST.f09_g16.20thC.125.02_TAUX_climo.nc'),
 PosixPath('/glade/scratch/br